In [3]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [23]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.mapping import generate_classbreaks
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.fires import aggregate
from georapid.formats import OutFormat

## Creating a client

**NOTE**

Ensure that `os.environ['x_rapidapi_key']` holds your Rapid API Key!


In [15]:
host = 'geofires.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

In [16]:
date_of_interest = datetime(2023, 8, 15)
aggregated_fires_dict = aggregate(client, date_of_interest, OutFormat.ESRI)
aggregated_fires_fset = FeatureSet.from_dict(aggregated_fires_dict)
aggregated_fires_sdf = aggregated_fires_fset.sdf
aggregated_fires_sdf

,index,count,OBJECTID,SHAPE
0,0,79,1,"{""rings"": [[[-13758824.165362816, 4537508.3427..."
1,2,16,2,"{""rings"": [[[-8562671.742656182, 4787508.34279..."
2,3,12,3,"{""rings"": [[[-9861709.848332841, 3537508.34279..."
3,4,23,4,"{""rings"": [[[-13758824.165362816, 5787508.3427..."
4,5,41,5,"{""rings"": [[[16552064.967092551, -4212491.6572..."
...,...,...,...,...
302,2605,1,303,"{""rings"": [[[-10511228.90117117, 5412508.34279..."
303,2606,2,304,"{""rings"": [[[-8129659.040763963, -3962491.6572..."
304,2608,3,305,"{""rings"": [[[-12459786.059686158, 7787508.3427..."
305,2611,2,306,"{""rings"": [[[-14408343.218201146, 9912508.3427..."


## Map news related to wildfires in California

In [22]:
def generate_aggregated_renderer(spatial_df, column='count'):
    alpha = 0.35
    class_breaks_renderer = generate_classbreaks(spatial_df, geometry_type='Polygon', field=column, method='esriClassifyNaturalBreaks', class_count=5, colors='YlOrRd', alpha=0.35)
    class_breaks_renderer['defaultSymbol']['color'][3] = int(alpha * 255)
    for class_break_info in class_breaks_renderer['classBreakInfos']:
        class_break_info['symbol']['color'][3] = int(alpha * 255)
    return class_breaks_renderer

def plot_aggregated(map_view, spatial_df, column='count'):
    """
    Plots the spatial dataframe as classified polygons using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        class_breaks_renderer = generate_aggregated_renderer(spatial_df)
        drawing_info = {
            'renderer': class_breaks_renderer
        }
        feature_collection = spatial_df.spatial.to_feature_collection(drawing_info=drawing_info)
        map_view.add_layer(feature_collection)

In [19]:
gis = GIS()

In [20]:
map_view = gis.map('California, USA')
map_view.basemap = 'osm'
map_view

MapView(layout=Layout(height='400px', width='100%'))

In [24]:
plot_aggregated(map_view, aggregated_fires_sdf)
map_view

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)